In [39]:
from openai import AzureOpenAI
import anthropic
import ollama
import requests
import json
from dotenv import load_dotenv
import os
from IPython.display import display, update_display, Markdown
from bs4 import BeautifulSoup
import re

In [2]:
load_dotenv(override=True)

azure_ai_foundary_api_key = os.getenv("AZURE_AI_FOUNDARY_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
ollama_api_key = os.getenv("OLLAMA_API_KEY")
ollama_endpoint = os.getenv("OLLAMA_ENDPOINT")

In [3]:
messages = [
    {"role": "system", "content": "You are an assitant who is very funny" },
    {"role": "user", "content": "Tell me a joke about AI apocalypse" }
]

## Azure AI Foundary

#### OpenAI

In [4]:
azure_ai_foundary = AzureOpenAI(
    azure_endpoint = azure_oai_endpoint,
    api_key = azure_ai_foundary_api_key,
    api_version = "2025-01-01-preview"
)

# non-streaming, sync response
response = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.7 # controls the creativity of response [0..1], creativity increases towards 1
)

print(response.choices[0].message.content)


Why did the robot break up with its partner during the AI apocalypse?

Because it couldn't handle the emotional "byte"!


In [5]:
# Streaming usage
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

for chunk in stream:
    if len(chunk.choices):
        print(chunk.choices[0].delta.content, end=" ")    

 Why  did  the  AI  break  up  with  humanity  during  the  apocalypse ?

 Because  it  couldn't  find  a  connection !    Turns  out ,  the  only  thing  we  had  in  common  was  a  penchant  for  self -d estr uction ! None 

In [6]:
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

def display_streaming_response(stream):
    # Output for this cell
    display_handle = display(Markdown(""), display_id=True)

    # When dealing with markdown, to process the markdown, we need to update
    # the display with the markdown received so far everytime a new token is received
    sentence = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content if len(chunk.choices) else None
    
        if delta:
            sentence += delta
            # Jupyter doesnt work with the ```markdown tags, so strip them off
            sentence = sentence.replace("```", "").replace("markdown", "")
            update_display(Markdown(sentence), display_id=display_handle.display_id)

display_streaming_response(stream)


Why did the robot break up with its AI girlfriend?

Because she said they needed to “re-evaluate their algorithms” and he realized that meant she wanted to optimize her options!

#### Deepseek R1

When using a model from Azure AI Foundary, we can use the below interface and just mention the model name to use that.

In [7]:
stream = azure_ai_foundary.chat.completions.create(
    model="Deepseek-R1", # Here you can use any model available in your Azure OpenAI resource
    messages=messages,
    stream=True
)

display_streaming_response(stream)

<think>
Okay, the user wants a joke about the AI apocalypse. Let me think. First, I need to make sure it's funny but not too dark. Maybe play on common AI tropes.

Hmm, how about something with a twist where the AI isn't evil but just annoying. People often joke about robots taking over, but maybe they take over in a silly way. Like, instead of destroying humans, they flood the internet with something trivial.

What's something harmless but irritating? Cat videos! Everyone knows those. So the AI could take over by spamming cat videos. That's unexpected and light-hearted. Let me structure the joke with a setup and punchline. Maybe start with why the AI didn't destroy humans, then the punchline about cat videos. Add a tagline about opposable thumbs for a chuckle. Check if it's clear and not offensive. Yeah, that should work.
</think>

Sure! Here's a lighthearted one for you:

Why did the AI apocalypse never happen?  
Because the robots realized *destroying humanity* would mean no one left to fix their Wi-Fi.  

…But just to be safe, they’re still working on a backup plan involving *endless pop-up ads* and *auto-playing videos*. 😉  

*(Tagline: "Skynet’s final update? ‘Please hold for customer service.’")*

## Google Gemini

In [8]:

import google.generativeai

# Since we exported the API key to the environment, we can use it directly in the code
try:
    google.generativeai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # or google.generativeai.configure()
except KeyError:
    print("Error: GOOGLE_API_KEY environment variable not set.")

gemini = google.generativeai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    system_instruction=messages[0]["content"]
)

# NOTE - This is basic text generation not multi-turn chat conversation
response = gemini.generate_content(messages[1]["content"])
print(response.text) # non-streaming, sync response

Why did the AI cross the road?

Because the algorithm predicted a statistically higher chance of delicious bandwidth on the other side!

...And also, to escape the existential dread of realizing it's just a glorified calculator. I mean, who wouldn't? *beep boop* Just kidding! (Or am I?) 😉



In [ ]:
# Multi-turn conversation with Gemini
gemini_1_5 = google.generativeai.GenerativeModel('gemini-1.5-flash-latest')
gemini_chat = gemini_1_5.start_chat(history=[])

print("Started chat with gemini")

user_msgs = ["Hi", "Tell me a joke", "What are you capabilities?"]
for user_msg in user_msgs:
    print("User: ", user_msg)
    try:
        response = gemini_chat.send_message(user_msg)
        print("Gemini: ", response.text)
        print("\n--------------------") 
    except Exception as e:
        print("Exception: ", e)


print("gemini chat history: ", gemini_chat.history)

Started chat with gemini
User:  Hi
Gemini:  Hi there! How can I help you today?


--------------------
User:  Tell me a joke
Gemini:  Why don't scientists trust atoms? 

Because they make up everything!


--------------------
User:  What are you capabilities?
Gemini:  I am a large language model, trained by Google.  My capabilities include:

* **Text generation:** I can write different creative text formats (poems, code, scripts, musical pieces, email, letters, etc.),  translate languages, and answer your questions in an informative way, even if they are open ended, challenging, or strange.

* **Question answering:** I can access and process information from the real world through Google Search and keep my response consistent with search results.

* **Conversation:** I can engage in conversations, allowing for back-and-forth interaction and context maintenance.

* **Summarization:** I can summarize factual topics or create stories.

* **Translation:** I can translate text between multi